Smurfette principle derivation, analysis of movie summaries, women are more often than not mentioned with a male character.

Process the summary, and count how many times each character is mentioned, use n-grams (2-3 ?) to see when they are mentioned together.

Maybe measure throughout the years ?

Or measure with different n for n-grams, interactive plot ? (possible to do both yearly + variable n too)

In [3]:
import pandas as pd
import json

In [10]:
# Character metadata
char_md_cols = ['Wikipedia movie ID',
'Freebase movie ID',
'Movie release date',
'Character name',
'Actor date of birth',
'Actor gender',
'Actor height (in meters)',
'Actor ethnicity (Freebase ID)',
'Actor name',
'Actor age at movie release',
'Freebase character/actor map ID',
'Freebase character ID',
'Freebase actor ID',
]
char_md = pd.read_csv('data/character.metadata.tsv', sep='\t', names=char_md_cols)
print('Number of characters:', char_md.shape[0])
char_md.head(2)

Number of characters: 450669


,Wikipedia movie ID,Freebase movie ID,Movie release date,Character name,Actor date of birth,Actor gender,Actor height (in meters),Actor ethnicity (Freebase ID),Actor name,Actor age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.62,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.78,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4


In [18]:
wikipedia_movie_IDS = list(summaries.keys())

In [60]:
import gzip
from xml.dom.minidom import parse, parseString


def open_parse_summary(wiki_id):
    # open and read gzipped xml file
    xml_file = gzip.open("./data/corenlp_plot_summaries/" + str(wiki_id) + ".xml.gz")
    document = parseString(xml_file.read())
    
    #Read all words
    words = dom.getElementsByTagName('word')
    #lemmas = dom.getElementsByTagName('lemma')
    
    #Looks for the desired node type and extracts their values
    words_list = [word.childNodes[0].nodeValue for word in words]
    #lemmas_list = [lemma.childNodes[0].nodeValue for lemma in lemmas]
    
    return words_list#, lemmas_list

In [ ]:
#TODO: RERUN THIS ON DESKTOP TO PROCESS ALL THE SUMMARIES
#i = 0
#tot = len(wikipedia_movie_IDS)
#
#for wiki_id in wikipedia_movie_IDS:
#    summary = list(open_parse_summary(wiki_id))[0]
#    with open("./data/corenlp_summaries_words/"+ wiki_id + ".txt", 'w') as fp:
#        for item in summary:
#            # write each item on a new line
#            fp.write("%s\n" % item)
#    i += 1
#    print(str(i) + "/" + str(tot))